In [1]:
# Use GPU option

# In Colab, go to Edit/Notebook Settings and choose the 'GPU' option before running this script

In [2]:
# Load data

# there are several ways to load data into Colab

# 1. Host your data to GitHub (up to 25MB) and use the url to the GitHub page
# e.g. df = pd.read_csv('https://raw.githubusercontent.com/junwang4/causal-language-use-in-science/master/data/pubmed_causal_language_use.csv') 

# 2. Host your data in your Google drive and then mount to your Google drive. You will be given an authorization code to finish the process
# e.g. the following code
# from google.colab import drive
# drive.mount('/drive')
# df = pd.read_csv('/drive/My Drive/train.tsv', sep='\t')

# 3. Upload your data to Colab Files. The uploaded file will be deleted when the session is disconnected. You will need to upload again after re-connect.
# e.g.   df = pd.read_csv('train.tsv', sep='\t')
# In this script we use method # 3

# We will use the Kaggle sentiment classification data
# https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews

In [18]:
import pandas as pd
from sklearn.metrics import f1_score

In [4]:
df = pd.read_csv('for_bert_eq_data_vrs.csv')


In [5]:
df.head()

,Unnamed: 0,tweet_text,tokenized_text,Subjectivity,polarity,nb_pronouns,nb_numerals,nb_exclm,nb_ques,count_modal_verbs,count_wh_words,nb_slang,label
0,0,RT @QuestionsNG: Ebola is not gonna come close...,"['RT', '@', 'QuestionsNG', ':', 'Ebola', 'gon'...",0.0,0.00,2,0,0,0,0,0,0,other_useful_information
1,1,It's too funny to watch MSNBC put an infectiou...,"['It', ""'s"", 'funny', 'watch', 'MSNBC', 'put',...",1.0,0.25,1,0,0,0,0,0,0,treatment
2,2,RT @mikecalma: How the #GOP fights infectious ...,"['RT', '@', 'mikecalma', ':', 'How', '#', 'GOP...",0.0,0.00,0,0,0,0,0,1,0,other_useful_information
3,3,@drbrainbugz it's all about spin! Guaranteed t...,"['@', 'drbrainbugz', ""'s"", 'spin', '!', 'Guara...",0.0,0.00,1,0,1,0,0,1,0,other_useful_information
4,4,WHO officially declares that #Senegal has ende...,"['WHO', 'officially', 'declares', '#', 'Senega...",0.0,0.00,1,0,0,0,0,1,0,disease_transmission


In [6]:
# prepare train and test data
# since fine tuning a BERT model still requires a significant amount of time, 
# only 1000 training examples and 1000 test examples will be used for demo purpose.
# prior experiment shows LinearSVC's best accuracy (3-fold CV) is about 62-65% depending on vectorization options
# BERT should be able to outperform LinearSVC with far fewer training examples.

dff = df.sample(frac=1)
#train_size = int(0.01 * len(dff))
#df_train = dff[:train_size]
#df_test = dff[train_size:]
df_train = dff[:1000]
df_test = dff[-1000:]
print(df_train.shape)
print(df_test.shape)
X_train, y_train = df_train['tweet_text'].values, df_train['label'].values
X_test, y_test = df_test['tweet_text'].values, df_test['label'].values

(1000, 13)
(1000, 13)


In [7]:
# check the category distribution in the train and test set

import numpy as np
unique, counts = np.unique(y_train, return_counts=True)
print(np.asarray((unique, counts)))
unique, counts = np.unique(y_test, return_counts=True)
print(np.asarray((unique, counts)))


[['affected_people' 'deaths_reports' 'disease_signs_or_symptoms'
  'disease_transmission' 'not_related_or_irrelevant'
  'other_useful_information' 'prevention' 'treatment']
 [181 22 109 118 72 308 74 116]]
[['affected_people' 'deaths_reports' 'disease_signs_or_symptoms'
  'disease_transmission' 'not_related_or_irrelevant'
  'other_useful_information' 'prevention' 'treatment']
 [188 20 123 104 69 281 89 126]]


In [8]:
# build a LinearSVC model as a baseline comparison to the BERT model
# since LinearSVC is a linear model, we can print out its top features in each category to see whether the model learned something meaningful
# in this example, the top 10 features for the "very negative" category (category 0) is printed out
# you can see that some top features are not so negative
# with only 1000 training examples, LinearSVC achieved 0.529 accuracy on the 1000 test examples.

from sklearn.feature_extraction.text import CountVectorizer
unigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=2, stop_words='english')
X_train_vec = unigram_count_vectorizer.fit_transform(X_train)
from sklearn.svm import LinearSVC
svm_clf = LinearSVC(C=1, max_iter=2000)
svm_clf.fit(X_train_vec,y_train)
feature_ranks = sorted(zip(svm_clf.coef_[0], unigram_count_vectorizer.get_feature_names()))
top_10 = feature_ranks[-10:]
print("Top 10 words")
for i in range(0, len(top_10)):
    print(top_10[i])
print()

Top 10 words
(0.8391286621991888, 'nurse')
(0.8478200866820932, 'eradicate')
(0.8488273899085251, 'nigerians')
(0.864753286376359, 'bats')
(0.8898818909923241, 'amarillo')
(0.94553115715031, 'improving')
(0.9663130982745137, 'comes')
(1.0619032776842743, '17')
(1.1836090347957806, '18')
(1.2879674396837437, 'case')



In [9]:
# to save some time, we are using a hold-out test to compare the LinearSVC and BERT models
# you can also try cross validation
# LinearSVC test accuracy
X_test_vec = unigram_count_vectorizer.transform(X_test)
y_pred = svm_clf.predict(X_test_vec)
svm_clf.score(X_test_vec,y_test)

0.656

In [10]:
# you can also try cross validation on the train set
# the result should not be far from the hold-out test
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
svm_pipe = Pipeline([('vect', unigram_count_vectorizer),('svm',LinearSVC(dual=True, max_iter=2000))])
scores = cross_val_score(svm_pipe,X_train,y_train,cv=3)
print(sum(scores)/len(scores))

0.6510042977108845


In [11]:
# print confusion matrix and classification report

from sklearn.metrics import confusion_matrix
y_pred = svm_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred)
print(cm)
print()

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

[[119   1  17   7   5  27   1  11]
 [  4   5   0   1   1   4   0   5]
 [ 24   0  81   5   0  11   0   2]
 [  4   0   4  72   2  14   5   3]
 [  0   0   3   0  29  29   2   6]
 [ 26   0   9  12  18 200   7   9]
 [  2   0   2   7   2  20  55   1]
 [  2   0   3   2   1  20   3  95]]

                           precision    recall  f1-score   support

          affected_people       0.66      0.63      0.64       188
           deaths_reports       0.83      0.25      0.38        20
disease_signs_or_symptoms       0.68      0.66      0.67       123
     disease_transmission       0.68      0.69      0.69       104
not_related_or_irrelevant       0.50      0.42      0.46        69
 other_useful_information       0.62      0.71      0.66       281
               prevention       0.75      0.62      0.68        89
                treatment       0.72      0.75      0.74       126

                 accuracy                           0.66      1000
                macro avg       0.68      0.59

In [12]:
# LinearSVC error analysis
# print out errors to check if any common patterns for further model improvement

# print out very positive examples that were predicted as negative
# you can print out different types of errors 
err_cnt = 0
for i in range(0, len(y_test)):
  if (y_test[i]==4 and y_pred[i]==1):
    print(X_test[i])
    err_cnt = err_cnt+1
print("errors:", err_cnt)

errors: 0


In [13]:
# install BERT sklearn wrapper written by charles9n
# check out the github page for fine tuning options and usage
# https://github.com/charles9n/bert-sklearn

!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 259 (delta 3), reused 3 (delta 0), pack-reused 247
Receiving objects: 100% (259/259), 519.36 KiB | 19.97 MiB/s, done.
Resolving deltas: 100% (125/125), done.
Processing /content/bert-sklearn
     |████████████████████████████████| 133kB 17.6MB/s 
     |████████████████████████████████| 6.9MB 35.6MB/s 
     |████████████████████████████████| 71kB 11.6MB/s 
  Created wheel for bert-sklearn: filename=bert_sklearn-0.3.1-cp36-none-any.whl size=54235 sha256=9b22a89b8110e7f028ec14377c3bba3d5fb82f13037a34abd482acc32fd43d3d
  Stored in directory: /root/.cache/pip/wheels/61/95/c6/5790aae8fb377f5ff356dbe58205aab28858595d6bff8197d0
Successfully built bert-sklearn
ERROR: botocore 1.19.25 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [14]:
# fine tune a BERT base uncased model
# since this wrapper has included vectorization using word embedding, no need to vectorize like in LinearSVC
# first the pre-trained BERT model will be loaded in
# then the training starts. 90% examples will be used as training examples and the other 10% as validation (parameter tuning)
# default setting is 3 epoch. Each epoch takes in some training data
from bert_sklearn import BertClassifier
model = BertClassifier()         # text/text pair classification
print(model)
model.fit(X_train, y_train)

100%|██████████| 231508/231508 [00:00<00:00, 16186549.71B/s]

Building sklearn text classifier...
BertClassifier(bert_config_json=None, bert_model='bert-base-uncased',
               bert_vocab=None, do_lower_case=None, epochs=3, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=1,
               ignore_label=None, label_list=None, learning_rate=2e-05,
               local_rank=-1, logfile='bert_sklearn.log', loss_scale=0,
               max_seq_length=128, num_mlp_hiddens=500, num_mlp_layers=0,
               random_state=42, restore_file=None, train_batch_size=32,
               use_cuda=True, validation_fraction=0.1, warmup_proportion=0.1)
Loading bert-base-uncased model...



100%|██████████| 433/433 [00:00<00:00, 125466.92B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 900, validation data size: 100



/usr/local/lib/python3.6/dist-packages/bert_sklearn/model/pytorch_pretrained/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  next_m.mul_(beta1).add_(1 - beta1, grad)




Epoch 1, Train loss: 1.9826, Val loss: 1.9080, Val accy: 34.00%





Epoch 2, Train loss: 1.5675, Val loss: 1.2728, Val accy: 52.00%





Epoch 3, Train loss: 0.9992, Val loss: 1.1058, Val accy: 64.00%



BertClassifier(bert_config_json=None, bert_model='bert-base-uncased',
               bert_vocab=None, do_lower_case=True, epochs=3, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=1,
               ignore_label=None,
               label_list=array(['affected_people', 'deaths_reports', 'disease_signs_or_symptoms',
       'disease_transmission', 'not_related_or_irrelevant',
       'other_useful_information', 'prevention', 'treatment'],
      dtype=object),
               learning_rate=2e-05, local_rank=-1, logfile='bert_sklearn.log',
               loss_scale=0, max_seq_length=128, num_mlp_hiddens=500,
               num_mlp_layers=0, random_state=42, restore_file=None,
               train_batch_size=32, use_cuda=True, validation_fraction=0.1,
               warmup_proportion=0.1)

In [15]:
model.save('bert-sentiment.model')

In [16]:
model.score(X_test, y_test)



Loss: 1.0269, Accuracy: 67.30%


67.30000000000001

In [19]:
y_pred = model.predict(X_test)
f1_score(y_pred, y_test, average='weighted')

0.6993571036652563

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = svm_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred)
print(cm)
print()

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
# BERT error analysis
err_cnt = 0
for i in range(0, len(y_test)):
  if (y_test[i]==4 and y_pred[i]==1):
    print(X_test[i])
    err_cnt = err_cnt+1
print("errors:", err_cnt)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model,X_train,y_train,cv=3)
print(sum(scores)/len(scores))